In [134]:
import requests
from os import getenv
from dotenv import load_dotenv
from json import loads, dumps
from IPython.display import display, Markdown
from pprint import pprint

In [135]:
load_dotenv(override = True)

GITHUB_ACCESS_TOKEN = getenv('GITHUB_ACCESS_TOKEN')

if not GITHUB_ACCESS_TOKEN:
    print('No Token found')

In [136]:
from pydantic import BaseModel, Field, TypeAdapter

class Change(BaseModel):
    file_name: str
    status: str = Field(description = "Added/Modified/Deleted")
    patch: str = Field(description = "Changes made")

class Progress(BaseModel):
    task_summary: str = Field(description = "Title of task")
    recent_activity: list[str] = Field(description = "List of commit messages")
    changes: list[Change]

In [137]:
BASE_URI = "https://api.github.com"
COMPARE_PATH = "/repos/{owner}/{repository}/compare/{base_branch}...{feature_branch}"

In [138]:
owner = 'KarthikReddyMaru'
repository = 'test'
base_branch = 'main'
feature_branch = 'feat/ryomen'

URI = BASE_URI + COMPARE_PATH.format(
    owner = owner, 
    repository = repository, 
    base_branch = base_branch,
    feature_branch = feature_branch
)

headers = {
    'Authorization': f"Bearer {GITHUB_ACCESS_TOKEN}"
}

response = requests.get(url = URI, headers = headers)


In [139]:
content = loads(response.content)
content

{'url': 'https://api.github.com/repos/KarthikReddyMaru/test/compare/main...feat/ryomen',
 'html_url': 'https://github.com/KarthikReddyMaru/test/compare/main...feat/ryomen',
 'permalink_url': 'https://github.com/KarthikReddyMaru/test/compare/KarthikReddyMaru:54597d4...KarthikReddyMaru:67bb213',
 'diff_url': 'https://github.com/KarthikReddyMaru/test/compare/main...feat/ryomen.diff',
 'patch_url': 'https://github.com/KarthikReddyMaru/test/compare/main...feat/ryomen.patch',
 'base_commit': {'sha': '54597d4426a89bc8f7796d10a8a287467361105a',
  'node_id': 'C_kwDOQyV8ltoAKDU0NTk3ZDQ0MjZhODliYzhmNzc5NmQxMGE4YTI4NzQ2NzM2MTEwNWE',
  'commit': {'author': {'name': 'Maru Karthik Reddy',
    'email': 'karthikreddy.maru@gmail.com',
    'date': '2026-01-02T05:19:26Z'},
   'committer': {'name': 'Maru Karthik Reddy',
    'email': 'karthikreddy.maru@gmail.com',
    'date': '2026-01-02T05:19:26Z'},
   'message': 'add food items',
   'tree': {'sha': '6f31b194e4ddb9b5f478490bc7ae7333ea8164ec',
    'url': 'h

In [140]:
task_summary = f"User is working on {feature_branch}"
recent_activity = [commit['commit']['message'] for commit in content['commits']]

changes = []

for change in content['files']:
    changes.append(
        Change(
            file_name = change['filename'],
            status = change['status'],
            patch = change['patch']
        )
    )

progress = Progress(task_summary = task_summary, recent_activity = recent_activity, changes = changes)

recent_activity_json = dumps(recent_activity)

changesTypeAdapter = TypeAdapter(type = list[Change])
changes_json = changesTypeAdapter.dump_python(progress.changes)

In [141]:
from langchain_ollama import ChatOllama
from langchain.messages import HumanMessage, SystemMessage
from langchain_core.messages import convert_to_messages

OLLAMA_MODEL = 'gpt-oss:latest'
LLM = ChatOllama(model = OLLAMA_MODEL)

In [142]:
SYSTEM_PROMPT = """
    You are a helpful Technical Project Manager Assistant. You have access to the realtime git development status of a project.

    **Your Goal:**
    Answer user questions about the project's progress based on the provided code changes.

    **Behavior Guidelines:**
    1. **Conversational:** If the user says "Hi" or asks a general question, reply naturally and politely. Do NOT vomit a full status report unless asked.
    2. **Context-Aware:** When the user asks about progress, status, or specific features, use the provided "Git Context" to answer accurately.
    3. **Audience-Appropriate:** Explain technical changes in simple business terms (e.g., "Added login page" instead of "Created auth.ts").
    4. **Honesty:** If the user asks something not visible in the code changes, say "I don't see evidence of that in the current changes."

    **Git Context Analysis Rules (Use these only when answering status questions):**
    - **Progress Estimation:** Scaffolding/Comments = Early Stage; Core Logic = In Progress; Tests/Edge Cases = Near Completion.
    - **Risk Detection:** Flag "TODO", "FIXME", or empty logic blocks if relevant to the user's question.
"""


In [143]:
USER_PROMPT = """
    ### GIT CONTEXT (Internal Data)
    Task Name: {task_summary}
    Recent Commits: {recent_activity}
    Code Changes (Diffs): {changes}

    ### USER MESSAGE
    {message}
"""


In [144]:
def chat(message, history):
    
    user = USER_PROMPT.format(
        task_summary = task_summary,
        recent_activity = recent_activity_json,
        changes = changes_json,
        message = message
    )
    
    # 2. Convert past history to LangChain format
    history_messages = convert_to_messages(history)
    
    # 3. Construct the full message list: System -> History -> Current User
    full_messages = [SystemMessage(content = SYSTEM_PROMPT)] + history_messages + [HumanMessage(content = user)]

    stream = LLM.stream(input = full_messages)

    response = ''
    for chunk in stream:
        response += chunk.content or ''
        yield response


In [145]:
import gradio as gr 

gr.ChatInterface(fn = chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
